# Experiments of stage 2

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import dgl

from dgl.data import CSVDataset

In [5]:
data_path = '~/.CEData/'

## Pre-process sampled data
* The sampled data shows what the input format of the stage 2 should be like:
---
* cell column: cell name
* time column: the time point when the cell data is sampled
* x, y, z columns: the 3-D positions
* size column: the volumn of the cell
---
* All input files of stage 2 should be aligned with the format here

In [27]:
raw_data = pd.read_csv(f'{data_path}raw-data-part-1a.csv')
raw_data.drop(['cellTime'], axis=1, inplace=True)
raw_data.drop(raw_data[(raw_data.time == 'time')].index, axis=0, inplace=True)
raw_data

,cell,time,x,y,z,size
0,AB,1,454,286,14,53
1,AB,2,494,244,14.5,68
2,AB,3,497,235,14,48
3,AB,4,491,223,14,50
4,ABa,5,519,272,14.3,70
...,...,...,...,...,...,...
252293,Cppp,87,578,205,14.1,40
252294,Cppp,88,583,215,14,40
252295,Cppp,89,573,210,13.1,38
252296,Cpppa,90,562,233,14.8,36


In [35]:
raw_data.time = raw_data.time.astype('float')
raw_data.x = raw_data.x.astype('float')
raw_data.y = raw_data.y.astype('float')
raw_data.z = raw_data.z.astype('float')
raw_data['size'] = raw_data['size'].astype('float')

## Build a DGL graph based on the given sampled data

0         53.0
1         68.0
2         48.0
3         50.0
4         70.0
          ... 
252293    40.0
252294    40.0
252295    38.0
252296    36.0
252297    36.0
Name: size, Length: 252285, dtype: float64